In [33]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import numpy as np

from mltu.inferenceModel import OnnxInferenceModel
from mltu.utils.text_utils import ctc_decoder, get_cer
from tqdm import tqdm

### Carregar o dataset

In [34]:
df = pd.read_csv('../data/data_20230622/data.csv', sep=';')

In [35]:
df.head()

,ID,Label,UserId,UserName,UserRole,Request,SheetId,SheetName,Date,Parent1Name,Parent2Name,Parent3Name
0,0,47,9b705e6cd6,Luis,PARTICIPANT,5f6ae87ce45a8d000156814a,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
1,1,26,0f1a3956d7,Matilde,PARTICIPANT,5f6ae88be45a8d000156814c,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
2,2,40,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
3,3,48,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN
4,4,41,ea5dee75ac,Lorenzo,PARTICIPANT,5f6ae8a3e45a8d000156814e,5f6a1c5aa7b11b0001722ac7,100er-Zahlenband II,Sat May 20 10:46:58 UTC 2023,2. Klasse,Niklas,NaN


In [36]:
df = df[df['UserRole'] == 'PARTICIPANTBR']
df.head()

,ID,Label,UserId,UserName,UserRole,Request,SheetId,SheetName,Date,Parent1Name,Parent2Name,Parent3Name
14881,14881,Natureza,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14882,14882,Planeta,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14883,14883,Terra,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14884,14884,Flor,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos
14885,14885,O nosso planeta é muito bonito,448905d42b34dbd,JOSE GABRIEL LIMA COSTA,PARTICIPANTBR,637d057802ae3800010957af,63276c6a8b87bc0001383e01,Avaliação da Escrita 2 Ano,Tue May 23 11:26:46 UTC 2023,2ºC,Emeb Myriam Penteado Rodrigues Alckmin,Ferraz de Vasconcelos


### Tratamento do dataset

In [37]:
# criar um novo dataframe com as colunas id e label
df = df[['ID', 'Label']]

In [38]:
# transformar a coluna id em string
df['ID'] = df['ID'].astype(str)

In [39]:
# adicionar uma string '../data/data_20230622/' no início da coluna id
df['ID'] = '../data/data_20230622/' + df['ID'] + '.jpeg'

In [40]:
# guardar esse novo dataframe em um arquivo csv
if not os.path.exists('labels_BR.csv'):    
    df.to_csv('labels.csv', index=False)
else :
    print('Arquivo já existe')    

Arquivo já existe


In [68]:
df.head()

,ID,Label
14881,../data/data_20230622/14881.jpeg,Natureza
14882,../data/data_20230622/14882.jpeg,Planeta
14883,../data/data_20230622/14883.jpeg,Terra
14884,../data/data_20230622/14884.jpeg,Flor
14885,../data/data_20230622/14885.jpeg,O nosso planeta é muito bonito


### Rodar o modelo

In [41]:
class ImageToWordModel(OnnxInferenceModel):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def predict(self, image: np.ndarray):
        image = cv2.resize(image, self.input_shape[:2][::-1])

        image_pred = np.expand_dims(image, axis=0).astype(np.float32)

        preds = self.model.run(None, {self.input_name: image_pred})[0]

        text = ctc_decoder(preds, self.vocab)[0]

        return text

In [42]:
model = ImageToWordModel("./Models/model.onnx")

In [69]:
accum_cer = []
for index, row in tqdm(df.iterrows()):
    image = cv2.imread(row['ID'])
    text = model.predict(image)
    cer, distance = get_cer(row['Label'], text)
    print(f'Label: {row["Label"]} - Text: {text} - CER: {cer} - Distance: {distance}')
    accum_cer.append(cer)
    if index == 14885:
        break

print(f'Mean CER: {np.mean(accum_cer)}')


4it [00:00, 269.25it/s]

Label: Natureza - Text: NATTA - CER: 1.4 - Distance: 7
Label: Planeta - Text: SAA - CER: 2.3333333333333335 - Distance: 7
Label: Terra - Text: PaMTA - CER: 1.0 - Distance: 5
Label: Flor - Text: AA - CER: 2.0 - Distance: 4
Label: O nosso planeta é muito bonito - Text: O OSSO TAAA ADA - CER: 0.9333333333333333 - Distance: 14
Mean CER: 1.5333333333333334
